In [ ]:
# /// CLEAR ALL OUTPUTS BEFORE PUSHING ///

import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx

# Charger le fichier GeoJSON des aménagements cyclables
gdf = gpd.read_file('data/amenagements-cyclables.geojson')

# Filtrer les données pour Paris en utilisant la colonne 'arrondissement'
gdf_paris = gdf[gdf['arrondissement'].notnull()]  # Filtrer pour Paris

# Reprojeter les données en Web Mercator pour correspondre au fond de carte
gdf_paris_web_mercator = gdf_paris.to_crs(epsg=3857)

# Créer une figure et un axe
fig, ax = plt.subplots(figsize=(15, 10))

# Tracer les aménagements cyclables
gdf_paris_web_mercator.plot(ax=ax, alpha=0.7, linewidth=1, column='amenagement', legend=True)

# Ajouter le fond de carte
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

# Titre et mise en page
plt.title('Aménagements cyclables à Paris', fontsize=16)
plt.axis('off')

# Ajuster les limites de la carte aux données de Paris
ax.set_xlim(gdf_paris_web_mercator.total_bounds[[0, 2]])
ax.set_ylim(gdf_paris_web_mercator.total_bounds[[1, 3]])

# Afficher la carte
plt.tight_layout()
plt.show()

# Quelques statistiques sur les aménagements cyclables à Paris
print(gdf_paris['amenagement'].value_counts())

# Calculer la longueur totale des aménagements cyclables (en mètres)
gdf_paris['longueur'] = gdf_paris.to_crs(epsg=2154).geometry.length
total_length = gdf_paris['longueur'].sum()
print(f"Longueur totale des aménagements cyclables à Paris : {total_length/1000:.2f} km")

# Sauvegarder les résultats pour Paris dans un nouveau fichier GeoJSON
gdf_paris.to_file('data/resultats_paris_cyclables.geojson', driver='GeoJSON')


In [ ]:
print(gdf.columns)

In [ ]:
# pip install contextily

In [ ]:
import geopandas as gpd
import folium
from tqdm import tqdm

# Charger le fichier GeoJSON des aménagements cyclables
gdf = gpd.read_file('data/amenagements-cyclables.geojson')

# Filtrer les données pour Paris en utilisant la colonne 'arrondissement'
gdf_paris = gdf[gdf['arrondissement'].notnull()]  # Filtrer pour Paris

# Supprimer les colonnes non nécessaires qui peuvent causer des problèmes de sérialisation
gdf_paris = gdf_paris.drop(columns=['date_export'])  # Supprimer la colonne problématique

# Filtrer pour ne garder que les lignes avec des géométries valides
gdf_paris = gdf_paris[gdf_paris.geometry.notnull()]

# Créer une carte centrée sur Paris
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Définir les couleurs pour chaque type d'aménagement
colors = {
    'Piste cyclable': 'green',
    'Voie verte': 'blue',
    'Bande cyclable': 'orange',
    'Autre': 'red'
}

# Fonction de style pour GeoJson
def style_function(feature):
    amenagement_type = feature['properties']['amenagement']
    color = colors.get(amenagement_type, 'black')
    return {
        'color': color,
        'weight': 2
    }

# Ajouter les différents types d'aménagements comme calques distincts avec une barre de progression
for amenagement_type in tqdm(gdf_paris['amenagement'].unique(), desc="Ajout des calques"):
    subset = gdf_paris[gdf_paris['amenagement'] == amenagement_type]
    
    # Vérifier si le sous-ensemble n'est pas vide avant d'ajouter à la carte
    if not subset.empty:
        folium.GeoJson(
            subset,
            name=amenagement_type,
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(fields=['amenagement', 'arrondissement']),
        ).add_to(m)

# Ajouter le contrôle des calques pour pouvoir sélectionner/désélectionner les types d'aménagements
folium.LayerControl().add_to(m)

# Ajouter une légende personnalisée
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 160px; height: auto; 
                background-color: white; border:2px solid black; z-index:9999; 
                font-size:12px; padding: 10px;">
    <b>Légende</b><br>
    <i style="background:green; width: 20px; height: 20px; display: inline-block; border:1px solid black;"></i> Piste cyclable<br>
    <i style="background:blue; width: 20px; height: 20px; display: inline-block; border:1px solid black;"></i> Voie verte<br>
    <i style="background:orange; width: 20px; height: 20px; display: inline-block; border:1px solid black;"></i> Bande cyclable<br>
    <i style="background:red; width: 20px; height: 20px; display: inline-block; border:1px solid black;"></i> Autre<br>
    </div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Enregistrer la carte dans un fichier HTML
m.save('data/carte_interactive_amenagements.html')

# Si tu veux afficher la carte dans un notebook Jupyter
m